In [7]:
import json

with open("assets/metadata/eikon_tickers.json") as f:
    instruments = json.load(f)

instruments

{'Asia': ['UNIB.XA',
  'HEZB.XA',
  'AHEK.XA',
  'ESHG.XA',
  'EVG1.XA',
  'MQER.XA',
  'ALMS.XA',
  'EJWG.XA',
  'ASEV.XA',
  'ALBH.BH',
  'ZAIN.BH',
  'EBRIT.BH',
  'UGHC.BH',
  'SPKL.BH',
  'APMTB.BH',
  'ABCB.BH',
  'SOLID.BH',
  'ARIG.BH',
  'AUBB.BH',
  'BANA.BH',
  'BARKA.BH',
  'BSRC.BH',
  'BTEL.BH',
  'BBKB.BH',
  'BCFC.BH',
  'BFMC.BH',
  'GHG.BH',
  'BISB.BH',
  'BKIC.BH',
  'BMEB.BH',
  'BMMI.BH',
  'BNHB.BH',
  'CNCO.BH',
  'PARK.BH',
  'DUTY.BH',
  'EICB.BH',
  'FMLY.BH',
  'GFHB.BH',
  'ITHMR.BH',
  'NASS.BH',
  'NATB.BH',
  'NHTL.BH',
  'DPCB.BH',
  'SALAM.BH',
  'SEEF.BH',
  'SICO.BH',
  'TAKA.BH',
  'INOV.BH',
  'GTFP.BH',
  'UGIC.BH',
  'KHCB.BH',
  'KPCL.DH',
  'RAKC.DH',
  'MSPM.DH',
  'GDMF.DH',
  'BEAC.DH',
  'ACFC.DH',
  'DESH.DH',
  'CROW.DH',
  'BARA.DH',
  'SALV.DH',
  'EBLN.DH',
  'MJLB.DH',
  'RDFO.DH',
  'ZHIN.DH',
  'GSPF.DH',
  'PADM.DH',
  'GPHI.DH',
  'GBBP.DH',
  'BSCC.DH',
  'UNIQ.DH',
  'AAMR.DH',
  'SAIH.DH',
  'GENN.DH',
  'ENVO.DH',
  'SICL.DH',

In [8]:
for region in instruments.keys():
    print(f"{region}: {len(instruments[region])}")

Asia: 30585
Africa: 1365
Americas: 16026
Europe: 12017
Oceania: 2198


In [1]:
import pandas as pd

In [47]:
df = pd.read_parquet(r"assets\Eikon\europe_data.parquet")
df['Company Name'] = df['Company Name'].replace("", None)
df['Company Name'] = df.groupby(['Instrument']).ffill()['Company Name']
df

,Instrument,Company Name,Country of Incorporation,Country of Exchange,Country of Headquarters,Instrument Type,ISIN,ISIN Code,ISIN_CODE,SEDOL,...,Trade Account Payables - Total,Advertising Expense,Labor & Related Expenses - Total,Dividend Per Share - Actual,"EPS - Basic - excl Extraordinary Items, Common - Total",TotAssets.Date,Price Close.Date,MktCap.Date,CompanySharesOutstanding.Date,Company Shares
0,AMAV.VI,AMAG Austria Metall AG,Austria,Austria,Austria,Ordinary Shares,AT00000AMAG3,AT00000AMAG3,AT00000AMAG3,B53JY51,...,112313000.0,<NA>,190066000.0,<NA>,3.09891,2022-12-31T00:00:00Z,2022-12-29T00:00:00Z,2022-12-31T00:00:00Z,2022-12-30T00:00:00Z,35264000.0
1,AMAV.VI,AMAG Austria Metall AG,,,,,,,,,...,107860000.0,<NA>,175154000.0,<NA>,1.85112,2021-12-31T00:00:00Z,2021-12-30T00:00:00Z,2021-12-31T00:00:00Z,2021-12-30T00:00:00Z,35264000.0
2,AMAV.VI,AMAG Austria Metall AG,,,,,,,,,...,59111000.0,<NA>,146618000.0,0.5,0.31287,2020-12-31T00:00:00Z,2020-12-30T00:00:00Z,2020-12-31T00:00:00Z,2020-12-30T00:00:00Z,35264000.0
3,AMAV.VI,AMAG Austria Metall AG,,,,,,,,,...,73050000.0,<NA>,157819000.0,2.4,1.09579,2019-12-31T00:00:00Z,2019-12-30T00:00:00Z,2019-12-31T00:00:00Z,2019-12-30T00:00:00Z,35264000.0
4,AMAV.VI,AMAG Austria Metall AG,,,,,,,,,...,89966000.0,<NA>,147939000.0,1.2,1.26307,2018-12-31T00:00:00Z,2018-12-28T00:00:00Z,2018-12-31T00:00:00Z,2018-12-28T00:00:00Z,35264000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,FSNT.DE,Fashionette AG,,,,,,,,,...,<NA>,<NA>,<NA>,<NA>,<NA>,2000-12-31T00:00:00Z,,2000-12-31T00:00:00Z,,<NA>
958,FSNT.DE,Fashionette AG,,,,,,,,,...,<NA>,<NA>,<NA>,<NA>,<NA>,1999-12-31T00:00:00Z,,1999-12-31T00:00:00Z,,<NA>
959,FSNT.DE,Fashionette AG,,,,,,,,,...,<NA>,<NA>,<NA>,<NA>,<NA>,1998-12-31T00:00:00Z,,1998-12-31T00:00:00Z,,<NA>
960,FSNT.DE,Fashionette AG,,,,,,,,,...,<NA>,<NA>,<NA>,<NA>,<NA>,1997-12-31T00:00:00Z,,1997-12-31T00:00:00Z,,<NA>


In [50]:
df[df['CUSIP Code'].notna()]['CUSIP Code']

52    77725109
Name: CUSIP Code, dtype: Int64

In [42]:
df[df['Instrument']=='AMAV.VI']

,Instrument,Company Name,Country of Incorporation,Country of Exchange,Country of Headquarters,Instrument Type,ISIN,ISIN Code,ISIN_CODE,SEDOL,...,Trade Account Payables - Total,Advertising Expense,Labor & Related Expenses - Total,Dividend Per Share - Actual,"EPS - Basic - excl Extraordinary Items, Common - Total",TotAssets.Date,Price Close.Date,MktCap.Date,CompanySharesOutstanding.Date,Company Shares
0,AMAV.VI,AMAG Austria Metall AG,Austria,Austria,Austria,Ordinary Shares,AT00000AMAG3,AT00000AMAG3,AT00000AMAG3,B53JY51,...,112313000.0,<NA>,190066000.0,<NA>,3.09891,2022-12-31T00:00:00Z,2022-12-29T00:00:00Z,2022-12-31T00:00:00Z,2022-12-30T00:00:00Z,35264000
1,AMAV.VI,AMAG Austria Metall AG,,,,,,,,,...,107860000.0,<NA>,175154000.0,<NA>,1.85112,2021-12-31T00:00:00Z,2021-12-30T00:00:00Z,2021-12-31T00:00:00Z,2021-12-30T00:00:00Z,35264000
2,AMAV.VI,AMAG Austria Metall AG,,,,,,,,,...,59111000.0,<NA>,146618000.0,0.5,0.31287,2020-12-31T00:00:00Z,2020-12-30T00:00:00Z,2020-12-31T00:00:00Z,2020-12-30T00:00:00Z,35264000
3,AMAV.VI,AMAG Austria Metall AG,,,,,,,,,...,73050000.0,<NA>,157819000.0,2.4,1.09579,2019-12-31T00:00:00Z,2019-12-30T00:00:00Z,2019-12-31T00:00:00Z,2019-12-30T00:00:00Z,35264000
4,AMAV.VI,AMAG Austria Metall AG,,,,,,,,,...,89966000.0,<NA>,147939000.0,1.2,1.26307,2018-12-31T00:00:00Z,2018-12-28T00:00:00Z,2018-12-31T00:00:00Z,2018-12-28T00:00:00Z,35264000
5,AMAV.VI,AMAG Austria Metall AG,,,,,,,,,...,77564000.0,<NA>,140232000.0,1.2,1.79106,2017-12-31T00:00:00Z,2017-12-29T00:00:00Z,2017-12-31T00:00:00Z,2017-12-29T00:00:00Z,35264000
6,AMAV.VI,AMAG Austria Metall AG,,,,,,,,,...,73322000.0,<NA>,132210000.0,1.2,1.3131,2016-12-31T00:00:00Z,2016-12-30T00:00:00Z,2016-12-31T00:00:00Z,2016-12-30T00:00:00Z,35264000
7,AMAV.VI,AMAG Austria Metall AG,,,,,,,,,...,55566000.0,<NA>,123020000.0,1.2,1.14942,2015-12-31T00:00:00Z,2015-12-30T00:00:00Z,2015-12-31T00:00:00Z,2015-12-30T00:00:00Z,35264000
8,AMAV.VI,AMAG Austria Metall AG,,,,,,,,,...,55428000.0,<NA>,117063000.0,1.2,1.67911,2014-12-31T00:00:00Z,2014-12-30T00:00:00Z,2014-12-31T00:00:00Z,2014-12-30T00:00:00Z,35264000
9,AMAV.VI,AMAG Austria Metall AG,,,,,,,,,...,47532000.0,<NA>,110654000.0,0.6,1.58882,2013-12-31T00:00:00Z,2013-12-30T00:00:00Z,2013-12-31T00:00:00Z,2013-12-30T00:00:00Z,35264000
